# Project 2

# 0 - Begin Spark

In [5]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    #.config("spark.executor.memory", "10g")
    #.config("spark.driver.memory", "10g")
    .getOrCreate()
)

## 1 - (Continued...)

### 1.9 - External dataset

#### 1.9.1 - Postcode + SA2 dataset

In [6]:
from pyspark.sql import functions as F

sa2_postcode_df = spark.read.csv('../data/SA2_to_Postcode/2023 Locality to 2021 SA2 Coding Index.csv', header = True, inferSchema = True)

# remove NaN postcodes
sa2_postcode_df = sa2_postcode_df.filter((sa2_postcode_df['POSTCODE'].isNotNull()) & (sa2_postcode_df['POSTCODE'] != 'NA'))
# cast POSTCODE col to int
sa2_postcode_df = sa2_postcode_df.withColumn('POSTCODE', sa2_postcode_df['POSTCODE'].cast('int'))

print(f'Number of entries = {sa2_postcode_df.count()}')
sa2_postcode_df = sa2_postcode_df.orderBy('POSTCODE', ascending = True)
sa2_postcode_df.limit(5)

Number of entries = 35901


LOCALITY_ID,LOCALITY_NAME,LOCALITY_TYPE,POSTCODE,STATE,SA2_MAINCODE_2021,SA2_NAME_2021
loc0072f7062952,DARWIN,ALIAS LOCALITY,800,NT,701011002,Darwin City
loc0072f7062952,DARWIN CITY,GAZETTED LOCALITY,800,NT,701011002,Darwin City
loc0072f7062952,LAMEROO BEACH,ALIAS LOCALITY,800,NT,701011002,Darwin City
loc0072f7062952,RAILWAY DAM,ALIAS LOCALITY,800,NT,701011002,Darwin City
loc7eba8173ea5b,ALAWA,GAZETTED LOCALITY,810,NT,701021010,Alawa


#### 1.9.2 - Merged Transactions dataset

In [7]:
merge_df_FINAL = spark.read.parquet('../data/merged/merge_df_E.parquet')
print(f'Number of entries = {merge_df_FINAL.count()}')
merge_df_FINAL.limit(5)

Number of entries = 14195717


user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,consumer_id,name,address,state,postcode,gender,consumer_fraud_order_datetime,consumer_fraud_probability,merchant_fraud_order_datetime,merchant_fraud_probability
14935,79417999332,136.06570809815838,23acbb7b-cf98-458...,2021-11-26,Phasellus At Company,"([gift, card, nov...",1059280,Cameron Adams,0280 Carpenter Lodge,QLD,4563,Male,NULL,NULL,NULL,NULL
1,46451548968,72.61581642788431,76bab304-fa2d-400...,2021-11-26,Tempus Eu Ligula ...,((heaLth and beau...,1195503,Yolanda Williams,413 Haney Gardens...,WA,6935,Female,NULL,NULL,NULL,NULL
14936,89518629617,3.0783487174439297,a2ae446a-2959-41c...,2021-11-26,Vulputate Velit E...,([tent and awnin...,986886,Maria Riley,416 Phillip View ...,SA,5157,Female,NULL,NULL,NULL,NULL
1,49167531725,51.58228625503599,7080c274-17f7-4cc...,2021-11-26,Felis Institute,[[digital goods: ...,1195503,Yolanda Williams,413 Haney Gardens...,WA,6935,Female,NULL,NULL,NULL,NULL
14936,31101120643,25.228114942417797,8e301c0f-06ab-45c...,2021-11-26,Commodo Hendrerit...,"[[cable, satellit...",986886,Maria Riley,416 Phillip View ...,SA,5157,Female,NULL,NULL,NULL,NULL


#### 1.9.3 - SA2 Population

In [96]:
population_df = spark.read.csv('../data/external_data/SA2_ERP.csv', header = True, inferSchema = True)
print(f'Number of entries = {population_df.count()}')
population_df.show(truncate = False)

Number of entries = 4908
+-------------------------------+---------------------------------+-----------------------------+---------------------------------------------+---------------+------------------------+---------+-----------------------------+------------------------------+--------------------------------+
|DATAFLOW                       |POP_COMP: Population Component   |REGION_TYPE: Region Type     |REGION: Region                               |FREQ: Frequency|TIME_PERIOD: Time Period|OBS_VALUE|UNIT_MEASURE: Unit of Measure|OBS_STATUS: Observation Status|OBS_COMMENT: Observation Comment|
+-------------------------------+---------------------------------+-----------------------------+---------------------------------------------+---------------+------------------------+---------+-----------------------------+------------------------------+--------------------------------+
|ABS:ERP_COMP_SA_ASGS2021(1.0.0)|10: Estimated Resident Population|SA2: Statistical Area Level 2|102011040: 

In [97]:
# Rename columns (ERP = Estimated Resident Population)
if ('REGION: Region' in population_df.columns):
    population_df = population_df.withColumnRenamed('REGION: Region', 'SA2_code')
if ('OBS_VALUE' in population_df.columns):
    population_df = population_df.withColumnRenamed('OBS_VALUE', 'ERP')

# Only select data from 2023 (more recent)
population_df = population_df.filter(F.col('TIME_PERIOD: Time Period') == 2023).select('SA2_code', 'ERP')
print(f'Number of entries = {population_df.count()}')
population_df.show(truncate = False)

Number of entries = 2454
+---------------------------------------------+-----+
|SA2_code                                     |ERP  |
+---------------------------------------------+-----+
|102011040: Umina - Booker Bay - Patonga      |24919|
|102021056: Warnervale - Wadalba              |21707|
|115021297: Dural - Kenthurst - Wisemans Ferry|22778|
|116011561: Blacktown - West                  |17538|
|116031316: Lethbridge Park - Tregear         |22239|
|117021637: Marrickville - South              |14397|
|117031648: Zetland                           |13968|
|118021567: Maroubra - South                  |11422|
|120031393: Croydon Park - Enfield            |17835|
|210011227: Essendon Airport                  |14   |
|211041269: Forest Hill                       |11023|
|211051285: Wandin - Seville                  |7989 |
|212021453: Endeavour Hills - North           |12048|
|212031558: Cranbourne East - South           |14472|
|212041565: Keysborough - North               |15119|
|21

In [98]:
population_df = population_df.withColumn('SA2_code', F.split(F.col('SA2_code'), ':').getItem(0))
population_df.limit(10)

SA2_code,ERP
102011040,24919
102021056,21707
115021297,22778
116011561,17538
116031316,22239
117021637,14397
117031648,13968
118021567,11422
120031393,17835
210011227,14


#### 1.9.4 - Income

In [99]:
income_df = spark.read.csv('../data/external_data/SA2_Income.csv', header = True, inferSchema = True)
print(f'Number of entries = {income_df.count()}')
income_df.limit(5)

Number of entries = 2017


OBJECTID,Statistical Areas Level 2 2021 code,Statistical Areas Level 2 2021 name,Area in square kilometres,Uniform Resource Identifier,Personal income: Employee income earners (no.) (Data year: 2020),Personal income: Employee income earners: Median age (years) (Data year: 2020),Personal income: Total employee income ($m) (Data year: 2020),Personal income: Median employee income ($) (Data year: 2020),Personal income: Mean employee income ($) (Data year: 2020),Personal income: Employee income as main source of income (%) (Data year: 2020),Personal income: Own unincorporated business income earners (no.) (Data year: 2020),Personal income: Own unincorporated business income earners: Median age (years) (Data year: 2020),Personal income: Total own unincorporated business income ($m) (Data year: 2020),Personal income: Median own unincorporated business income ($) (Data year: 2020),Personal income: Mean own unincorporated business income ($) (Data year: 2020),Personal income: Own unincorporated business income as main source of income (%) (Data year: 2020),Personal income: Investment income earners (no.) (Data year: 2020),Personal income: Investment income earners: Median age (years) (Data year: 2020),Personal income: Total investment income ($m) (Data year: 2020),Personal income: Median investment income ($) (Data year: 2020),Personal income: Mean investment income ($) (Data year: 2020),Personal income: Investment income as main source of income (%) (Data year: 2020),Personal income: Superannuation and annuity income earners (no.) (Data year: 2020),Personal income: Superannuation and annuity income earners: Median age (years) (Data year: 2020),Personal income: Total superannuation and annuity income ($m) (Data year: 2020),Personal income: Median superannuation and annuity income ($) (Data year: 2020),Personal income: Mean superannuation and annuity income ($) (Data year: 2020),Personal income: Superannuation and annuity income as main source of income (%) (Data year: 2020),Personal income: Total income earners (excl. Government pensions and allowances)(no.) (Data year: 2020),Personal income: Total income earners (excl. Government pensions and allowances): Median age (years) (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances) ($m) (Data year: 2020),Personal income: Median total income (excl. Government pensions and allowances) ($) (Data year: 2020),Personal income: Mean total income (excl. Government pensions and allowances) ($) (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): p80/P20 ratio (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): p80/P50 ratio (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): p20/P50 ratio (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): p10/P50 ratio (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Gini coefficient (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Income share of top 1% of earners (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Income share of top 5% of earners (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Income share of top 10% of earners (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Lowest Quartile (%) (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Second Quartile (%) (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Third Quartile (%) (Data year: 2020),Personal income: Total income (excl. Government pensions and allowances): Highest Quartile (%) (Data year: 2020),Government pensions and allowances: Age pension (no.) (Data year: 2023),Government pensions and allowances: Carer paym

In [100]:
income_df = income_df.select("Statistical Areas Level 2 2021 code", 
                             "`Personal income: Total income (excl. Government pensions and allowances) ($m) (Data year: 2020)`", 
                             "`Personal income: Median total income (excl. Government pensions and allowances) ($) (Data year: 2020)`", 
                             "`Personal income: Mean total income (excl. Government pensions and allowances) ($) (Data year: 2020)`", 
                             "`Personal income: Total income (excl. Government pensions and allowances): Gini coefficient (Data year: 2020)`")

# Rename columns
# SA2 code
if (income_df.columns[0] in income_df.columns):
    income_df = income_df.withColumnRenamed(income_df.columns[0], 'SA2_code')
# Total income
if (income_df.columns[1] in income_df.columns):
    income_df = income_df.withColumnRenamed(income_df.columns[1], 'total_income_($M)')
# Median total income
if (income_df.columns[2] in income_df.columns):
    income_df = income_df.withColumnRenamed(income_df.columns[2], 'median_income')
# Mean total income
if (income_df.columns[3] in income_df.columns):
    income_df = income_df.withColumnRenamed(income_df.columns[3], 'mean_income')
# Gini coefficient
if (income_df.columns[4] in income_df.columns):
    income_df = income_df.withColumnRenamed(income_df.columns[4], 'gini_coef')

print(f'Number of entries = {income_df.count()}')
income_df.limit(5)

Number of entries = 2017


SA2_code,total_income_($M),median_income,mean_income,gini_coef
101021007,149.8,44246,61745,0.574
101021008,345.3,62946,67345,0.363
101021009,456.0,61724,67276,0.366
101021010,241.1,64010,71770,0.383
101021012,739.7,75987,88257,0.411


#### 1.9.5 - Income with Population

In [105]:
# merge on SA2_code
pop_income_df = population_df.join(F.broadcast(income_df),
                                           population_df.SA2_code == income_df.SA2_code,
                                           how = 'left'
                                           ).drop(income_df.SA2_code)

print(f'Number of entries = {pop_income_df.count()}')
pop_income_df = pop_income_df.orderBy('ERP', ascending = False)
pop_income_df.limit(20)

Number of entries = 2454


SA2_code,ERP,total_income_($M),median_income,mean_income,gini_coef
116021632,33165,NULL,NULL,NULL,NULL
210051445,31976,NULL,NULL,NULL,NULL
209041437,30758,580.9,51609,57862,0.383
213051583,29321,NULL,NULL,NULL,NULL
507051313,28896,NULL,NULL,NULL,NULL
205031093,28254,683.1,41773,51152,0.475
213051468,28144,692.8,54788,61509,0.392
212011552,27999,NULL,NULL,NULL,NULL
404031107,27709,914.0,48980,57415,0.432
213041359,27259,321.7,54381,59009,0.393


#### 1.9.6 - Merge SA2 with Income

In [90]:
sa2_pop_income_df = sa2_postcode_pop_df.join(F.broadcast(income_df),
                                           sa2_postcode_df.SA2_MAINCODE_2021 == income_df.SA2_code,
                                           how = 'left'
                                           ).drop(income_df.SA2_code)

print(f'Number of entries = {sa2_pop_income_df.count()}')
sa2_pop_income_df = sa2_pop_income_df.orderBy('SA2_MAINCODE_2021')
sa2_pop_income_df.limit(20)

Number of entries = 35901


LOCALITY_ID,LOCALITY_NAME,LOCALITY_TYPE,POSTCODE,STATE,SA2_MAINCODE_2021,SA2_NAME_2021,ERP,total_income_($M),median_income,mean_income,gini_coef
loc4ed8f3504d55,BERLANG,GAZETTED LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc4ad93633108f,KAIN,ALIAS LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc820e6248e42d,BOMBAY,GAZETTED LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc7cc57252f0ca,CULLULLA,ALIAS LOCALITY,2580,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
locb094042a0875,BORO,GAZETTED LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc3d506f923c08,MAYFIELD,GAZETTED LOCALITY,2580,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc15bfcc3ea3ce,BRAIDWOOD,GAZETTED LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc08094a25c0b1,BENDOURA,GAZETTED LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
loc246c39b10b24,BRAIDWOOD,ALIAS LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574
locd86cc621ba4c,CORANG,GAZETTED LOCALITY,2622,NSW,101021007,Braidwood,4396,149.8,44246,61745,0.574


#### 1.9.X - All SA2 within a POA

In [83]:
# group by POSTCODE and aggregate SA2_MAINCODE_2021
postcode_grouped_sa2_df = sa2_pop_income_df.groupBy('POSTCODE').agg(
    # name only here for general idea of where the location is
    F.first('LOCALITY_NAME').alias('LOCALITY_NAME'),
    F.first('STATE').alias('STATE'),
    F.collect_set('SA2_MAINCODE_2021').cast('array<int>').alias('SA2_CODES'),
    F.collect_list('total_income_($M)'),
    F.collect_list('median_income'),
    F.collect_list('mean_income'),
    F.collect_list('gini_coef')
)

postcode_grouped_sa2_df.show(truncate = False)

+--------+---------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
postcode_grouped_sa2_df.printSchema()

root
 |-- POSTCODE: integer (nullable = true)
 |-- SA2_NAME: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- SA2_CODES: array (nullable = false)
 |    |-- element: integer (containsNull = true)

